In [3]:
import pandas as pd

# Load dataset
file_path = "F:/casts/dataset_rt.csv"
df = pd.read_csv(file_path)

# Ask user for number and bin
num = float(input("Enter number (e.g., 864.9): "))
bin_value = int(input("Enter bin value (e.g., 25): "))

# Build column name: zero-padded 5 digits
col_num = int((num - 600) * 10)
col_name = "cast_" + str(col_num).zfill(5)
print(f"Using column: {col_name}")

# Filter dataset by bin
df_bin = df[df["bin"] == bin_value]

if col_name not in df_bin.columns:
    print(f"Column '{col_name}' not found in dataset.")
else:
    # Group by target and calculate mean
    averages = df_bin.groupby("target")[col_name].mean()

    for target in [0, 1, 2, 3]:
        val = averages.get(target, "No data")
        print(f"Target {target}: {val}")


Using column: cast_02649
Target 0: 7976102.5227272725
Target 1: 8993313.05
Target 2: 9194878.228365384
Target 3: 3825926.849137931


In [5]:
import os
import pandas as pd
import numpy as np

# --- Config ---
FILE_PATH = r"F:/casts/dataset_rt.csv"   # update if needed
OUT_DIR   = os.path.dirname(FILE_PATH) or "."

# --- Load data ---
df = pd.read_csv(FILE_PATH)

# --- Inputs ---
bin_value = float(input("Enter bin value (e.g., 25): ").strip())
nums_str = input("Enter numbers separated by commas (e.g., 864.9, 865.2, 900): ").strip()

# Parse numbers
nums = [float(x) for x in nums_str.split(",") if x.strip()]

# Convert to cast column names:
# 1) (num - 600) * 10
# 2) int(...)  (truncate toward zero)
# 3) zero-pad to 5 digits and prefix "cast_"
def to_cast_col(n: float) -> str:
    col_num = int((n - 600) * 10)
    return "cast_" + str(col_num).zfill(5)

cast_cols = [to_cast_col(n) for n in nums]
print("Resolved columns:", ", ".join(cast_cols))

# --- Filter by bin ---
df_bin = df[df["bin"] == bin_value].copy()

if df_bin.empty:
    print(f"No rows found for bin == {bin_value}.")
    # Still produce an empty table with requested columns and targets 0-3
    result = pd.DataFrame(index=[0,1,2,3], columns=cast_cols, dtype=float)
else:
    # Ensure target is present; if not, raise a helpful message
    if "target" not in df_bin.columns:
        raise KeyError("Column 'target' not found in the dataset.")

    # Prepare an empty result table (targets 0..3 x requested columns)
    result = pd.DataFrame(index=[0,1,2,3], columns=cast_cols, dtype=float)

    # Compute means per target for each requested cast column
    for col in cast_cols:
        if col in df_bin.columns:
            means = df_bin.groupby("target")[col].mean()
            # Place into result, ensuring targets 0..3 (NaN if missing)
            result[col] = means.reindex([0,1,2,3])
        else:
            # Column not found; leave as NaN and note it
            print(f"Warning: column '{col}' not found in dataset.")

# --- Pretty print ---
print("\nAverage values by target (rows) and cast columns (columns):")
print(result)

# --- Save to CSV ---
safe_bin = str(bin_value).replace(".", "_")
out_path = os.path.join(OUT_DIR, f"averages_by_target_bin_{safe_bin}.csv")
result.to_csv(out_path, index_label="target")
print(f"\nSaved table to: {out_path}")


Resolved columns: cast_01502, cast_00067, cast_01642, cast_01422, cast_01512

Average values by target (rows) and cast columns (columns):
     cast_01502    cast_00067    cast_01642    cast_01422    cast_01512
0  2.133233e+07  1.300692e+06  3.745564e+06  1.138287e+07  8.940805e+06
1  1.992571e+07  1.461214e+06  4.742691e+06  1.070613e+07  8.878687e+06
2  1.484272e+07  1.129760e+06  6.238774e+06  1.118276e+07  7.049923e+06
3  3.894499e+07  1.892069e+06  3.287128e+06  1.106866e+07  1.454423e+07

Saved table to: F:/casts\averages_by_target_bin_5_0.csv
